# Setup & Import


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pvlib
import json
import os
from pvlib.pvsystem import PVSystem, Array, FixedMount
from pvlib.location import Location
from pvlib.modelchain import ModelChain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
import plotly.graph_objects as go
import plotly.io as pio
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
import forestci as fci
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import threading
from sklearn.metrics import make_scorer
import dash
from dash import dcc, html
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import webbrowser
from threading import Timer
import requests

import numpy as np

from sklearn.utils.parallel import Parallel, delayed
from sklearn.utils.validation import (
    check_is_fitted,
)
from sklearn.ensemble._base import _partition_estimators

pio.renderers.default = "browser"  # render plotly figures in browser

PARENT_DATA_DIR = os.getenv('PARENT_DATA_DIR')
if PARENT_DATA_DIR is None:
    raise ValueError("PARENT_DATA_DIR environment variable is not set")


dataDirpath = PARENT_DATA_DIR + r"\PRiOT\dataExport_3400_daily"  # "/Applications/Documents/TM Maxime/dataExport_3400_daily"#
dataCacheDirpath = os.path.join(dataDirpath, "cache")
logsDirpath = "../logs"
useCached = False
forceTrain = True
tuneMaxProductionEstimators = True
random_state = 42

if not os.path.exists(logsDirpath):
    os.makedirs(logsDirpath)

# Serializer


In [ ]:
# https://scikit-learn.org/stable/model_persistence.html
import pickle
import joblib
import os


class ModelSerializer:
    def _save_model(self, model, serial_type, save_params):
        serial_type.dump(model, save_params)

    def _retrieve_model(self, serial_type, retrieve_params):
        return serial_type.load(retrieve_params)


# save_model_path = "Serialized_models\\"


class JoblibSerializer(ModelSerializer):
    def save_model(self, model, save_model_path, filename):
        super()._save_model(model, joblib, os.path.join(save_model_path, filename + ".joblib"))

    def retrieve_model(self, save_model_path, filename):
        return super()._retrieve_model(joblib, os.path.join(save_model_path, filename + '.joblib'))


class PickleSerializer(ModelSerializer):
    def save_model(self, model, save_model_path, filename):
        with open(os.path.join(save_model_path, filename + ".pkl"), 'wb') as f:
            super()._save_model(model, pickle, f)

    def retrieve_model(self, save_model_path, filename):
        with open(os.path.join(save_model_path, filename + ".pkl"), 'rb') as f:
            return super()._retrieve_model(pickle, f)

# Functione

In [ ]:
def get_altitude_from_wgs84(longitude, latitude):
    # Convert WGS84 to LV95
    lv95_url = "https://geodesy.geo.admin.ch/reframe/wgs84tolv95"
    params_lv95 = {
        "easting": longitude,
        "northing": latitude,
        "format": "json"
    }
    
    response_lv95 = requests.get(lv95_url, params=params_lv95)
    if response_lv95.status_code != 200:
        raise Exception("Error converting WGS84 to LV95: " + response_lv95.text)
    
    lv95_data = response_lv95.json()
    lv95_easting = lv95_data["easting"]
    lv95_northing = lv95_data["northing"]
    
    # Get altitude from LV95 coordinates
    altitude_url = "https://api3.geo.admin.ch/rest/services/height"
    params_altitude = {
        "easting": lv95_easting,
        "northing": lv95_northing
    }
    
    response_altitude = requests.get(altitude_url, params=params_altitude)
    if response_altitude.status_code != 200:
        raise Exception("Error retrieving altitude: " + response_altitude.text)
    
    altitude_data = response_altitude.json()
    altitude = altitude_data["height"]
    
    return float(altitude)

## Import metadata


In [ ]:
metadataFilepath = os.path.join(dataDirpath, "metadata.json")

with open(metadataFilepath, 'r') as f:
    systemsMetadata = json.load(f)

# Add altitude to metadata, if not already present (TODO : imporove with multi threading)

for systemId, systemMetadata in tqdm(systemsMetadata.items()):
    if "loc_altitude" not in systemMetadata['metadata']:
        if "loc_longitude" in systemMetadata['metadata'] and "loc_latitude" in systemMetadata['metadata']:
            systemMetadata['metadata']["loc_altitude"] = get_altitude_from_wgs84(systemMetadata['metadata']["loc_longitude"], systemMetadata['metadata']["loc_latitude"])

# Save metadata with altitude
with open(metadataFilepath, 'w') as f:
    json.dump(systemsMetadata, f, indent=4)

## Import measures


In [ ]:
cacheFilename_systemsData_MeasuredDailyEnergy = os.path.join(dataCacheDirpath, 'systemsData_MeasuredDailyEnergy.pkl')
if useCached and os.path.exists(cacheFilename_systemsData_MeasuredDailyEnergy):
    print(f"Loading cached data in {cacheFilename_systemsData_MeasuredDailyEnergy}")
    systemsData_MeasuredDailyEnergy = pd.read_pickle(cacheFilename_systemsData_MeasuredDailyEnergy)
    systemsName_Valid = systemsData_MeasuredDailyEnergy.columns
else:
    # Load all csv files from the data directory
    systemsData = {}
    for file in os.listdir(dataDirpath):
        if file.endswith(".csv"):
            systemName = file.split("_")[0]
            systemsData[systemName] = pd.read_csv(os.path.join(dataDirpath, file))
            systemsData[systemName]['Datetime'] = pd.to_datetime(systemsData[systemName]['Timestamp'], unit='ms', utc=True).dt.tz_convert('Europe/Zurich')
            systemsData[systemName]['Date'] = (systemsData[systemName]['Datetime'] + pd.Timedelta(hours=1)).dt.date  # Convert the datetime to only the date, as the production is the daily production. The +1h is to manage the saving time. Normally PRiOT exports the data at midnight (local time) for the day after (e.g. the energy for the July 1st is saved at July 1st 00:00 Europe/Zurich). However it seams that the saving time is not always correctly handled, and sometime the export is done at 23:00 the day before (e.g. the energy for the July 1st is saved at June 30th 23:00 Europe/Zurich). This is why we add 1h to the datetime to be sure to have the correct date.

    systemsName = list(systemsData.keys())

    df_duplicate_list = list()
    for systemName, systemData in systemsData.items():
        # Save duplicate dates to log list, and the in a log file
        duplicates = systemData[systemData['Date'].duplicated(keep=False)]
        if len(duplicates) > 0:
            df_duplicate_list.append(duplicates)

            # Remove duplicate date where tt_forward_active_energy_total_toDay is the smallest
            # TODO maybe we should sum the energy of the duplicates instead of removing the smallest one. However, when looking in PRiOT Portal, it seams that in the daily energy, only the biggest value is represented. We do the same here.
            systemData.sort_values('tt_forward_active_energy_total_toDay', ascending=True, inplace=True)
            systemsData[systemName].drop_duplicates(subset='Date', keep='last', inplace=True)

        # Set date as the index and sort the data by date
        systemsData[systemName].set_index('Date', inplace=True)
        systemData.sort_index(ascending=True, inplace=True)

    # Save duplicate dates to log file
    df_duplicate = pd.concat(df_duplicate_list)
    print(f"Number of duplicate dates found: {len(df_duplicate)} (see log file for more details)")
    df_duplicate.to_csv(os.path.join(logsDirpath, 'duplicateDates.csv'), index=True)

    ## ----------------------------------------------- ##
    ## Convert data & Filter out invalid PRiOT systems ##
    ## ----------------------------------------------- ##

    systemsName_Valid = systemsName.copy()
    for systemName in systemsName:
        missingData = False
        # Check if the system has measures
        if len(systemsData[systemName]) == 0:
            missingData = True
            print(f"System {systemName} : No measures found")
        # Check if the system has metadata
        if systemName not in systemsMetadata:
            missingData = True
            print(f"System {systemName} : No metadata found")
        
        else:
            # Check metadata for the system
            for key in ['loc_latitude', 'loc_longitude', 'loc_altitude', 'pv_kwp']:
                # test that the key is present
                if key not in systemMetadata['metadata']:
                    missingData = True
                    print(f"System {systemName} : No '{key}' found")
                # if present, convert the value to a number, if possible
                elif not isinstance(systemsMetadata[systemName]['metadata'][key], (int, float)):
                    try:
                        systemsMetadata[systemName]['metadata'][key] = int(systemsMetadata[systemName]['metadata'][key])
                    except ValueError:
                        try:
                            systemsMetadata[systemName]['metadata'][key] = float(systemsMetadata[systemName]['metadata'][key])
                        except ValueError:
                            missingData = True
                            print(f"System {systemName} : The key-value '{key}:{systemsMetadata[systemName]['metadata'][key]}' is not a number")

            # Check metadata for the arrays
            if (len(systemsMetadata[systemName]['arrays']) == 0):
                print(f"System {systemName} : No PV arrays found")
                missingData = True
            else:
                for array_num, arrayData in systemsMetadata[systemName]['arrays'].items():
                    for key in ['pv_tilt', 'pv_azimut', 'pv_wp', 'pv_number']:
                        if key not in arrayData:
                            missingData = True
                            print(f"System {systemName} : No '{key}' found for array {array_num}")
                        # test that the value is a number
                        elif not isinstance(arrayData[key], (int, float)):
                            try:
                                arrayData[key] = int(arrayData[key])
                            except ValueError:
                                try:
                                    arrayData[key] = float(arrayData[key])
                                except ValueError:
                                    missingData = True
                                    print(f"System {systemName} : The key-value '{key}:{arrayData[key]}' is not a number for array {array_num}")
            
            # add the loss metadata if not present
            if 'loss' not in systemsMetadata[systemName]['metadata']:
                systemsMetadata[systemName]['metadata']['loss'] = 0

        if missingData:
            systemsName_Valid.remove(systemName)
            print(f"-> Removing system {systemName} from the list of systems")

    print(f"Number of systems with all the necessary data: {len(systemsName_Valid)}/{len(systemsName)}")

    # Filter out systems with less than 100 days of data
    minimumDays = 7
    for systemName in systemsName_Valid[:]:  # Create a copy of the list using slicing [:] to avoid removing elements while iterating over the list itself
        if len(systemsData[systemName]) < minimumDays:
            systemsName_Valid.remove(systemName)
            print(f"-> Removing system {systemName} from the list of systems because it has less than {minimumDays} days of data")

    print(f"Number of systems with at least {minimumDays} days of data: {len(systemsName_Valid)}/{len(systemsName)}")

    ## ---------------------------------------------------------------------------- ##
    ## Create one 2D DataFrame with the daily production of every remaining systems ##
    ## ---------------------------------------------------------------------------- ##

    # Create an empty list to store all measured data for each systems
    systemsData_MeasuredDailyEnergy_List = []

    # Iterate over each key-value pair in the systemsData dictionary
    for systemName in systemsName_Valid:
        # Extract the 'tt_forward_active_energy_total_toDay' column from the current dataframe
        measuredDailyEnergy = systemsData[systemName]['tt_forward_active_energy_total_toDay']

        # Rename the column with the system name
        measuredDailyEnergy.rename(systemName, inplace=True)

        systemsData_MeasuredDailyEnergy_List.append(measuredDailyEnergy)
        # Concatenate the column to the new_dataframe

    # Concatenate all the columns in the list to create one dataframe
    systemsData_MeasuredDailyEnergy = pd.concat(systemsData_MeasuredDailyEnergy_List, axis=1)
    systemsData_MeasuredDailyEnergy.index = pd.to_datetime(systemsData_MeasuredDailyEnergy.index)
    systemsData_MeasuredDailyEnergy.sort_index(inplace=True)

    ## ------------------ ##
    ## Save the dataframe ##
    ## ------------------ ##
    # Save the dataframe for later use
    systemsData_MeasuredDailyEnergy.to_pickle(cacheFilename_systemsData_MeasuredDailyEnergy)

# Print the dataframe
systemsData_MeasuredDailyEnergy

### Show missing value


## Choose system to train


In [ ]:
systemsName_Target = systemsName_Valid.copy()
# systemsName_Target = ["a001035"]

## Max production estimator


### Functions


In [ ]:
# Convert the power production with a given frequency to the total daily energy
def daily_energy(df_power):
    # Get the frequency in minutes
    freq_in_minutes = pd.Timedelta(df_power.index.freq).seconds / 60
    # Convert power from kW to kWh
    df_energy = df_power * (freq_in_minutes / 60)
    # Resample to daily frequency and sum the values
    daily_energy = df_energy.resample('D').sum()
    # daily_energy.index = daily_energy.index.date

    return daily_energy

# Simulate the daily production of a system from a start date to an end date using the given PVLib ModelChain


def generate_max_production_estimate(startDate, endDate, estimator: ModelChain, samplingFreq='1h'):
    # The end date is included in the simulation (end date at 23:59).
    # So we add 1 day to the end date to include the entire end date in the date_range(), and then we exclude the last value (end date +1 at 00:00) in the date_range().
    # TODO It is possible to take into account the horizon, using this method: https://pvlib-python.readthedocs.io/en/stable/gallery/shading/plot_simple_irradiance_adjustment_for_horizon_shading.html
    endDate = endDate + pd.Timedelta(days=1)

    times = pd.date_range(start=startDate, end=endDate, freq=samplingFreq, tz=estimator.location.tz, inclusive='left')
    weatherClearSky = estimator.location.get_clearsky(times)  # In W/m2
    estimator.run_model(weatherClearSky)
    production = estimator.results.ac / 1000  # Convert W to kW
    dailyProduction = daily_energy(production)
    dailyProduction.index = pd.to_datetime(dailyProduction.index.date)
    return dailyProduction

def generate_max_production_estimator(systemMetadata):
    latitude = systemMetadata['metadata']['loc_latitude']
    longitude = systemMetadata['metadata']['loc_longitude']
    altitude = systemMetadata['metadata']['loc_altitude']
    Wp_Tot = systemMetadata['metadata']['pv_kwp'] * 1000
    loss = systemMetadata['metadata']['loss'] * 100

    arrays = []
    for array_num, arrayData in systemMetadata['arrays'].items():
        array = Array(
            mount=FixedMount(surface_tilt=arrayData['pv_tilt'], surface_azimuth=arrayData['pv_azimut'], racking_model='open_rack'),
            module_parameters={'pdc0': arrayData['pv_wp'], 'gamma_pdc': -0.004},
            module_type='glass_polymer',
            modules_per_string=arrayData['pv_number'],
            strings=1,
            temperature_model_parameters=TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer'],
        )
        arrays.append(array)

    location = Location(latitude=latitude, longitude=longitude, altitude=altitude, tz='Europe/Zurich')
    system = PVSystem(arrays=arrays, 
                        inverter_parameters={'pdc0': Wp_Tot, 'eta_inv_nom': 0.96},
                        losses_parameters = {'nameplate_rating': loss, 'soiling': 0, 'shading': 0, 'snow': 0, 'mismatch': 0, 'wiring': 0, 'connections': 0, 'lid': 0, 'age': 0, 'availability': 0})
    modelChain = ModelChain(system, location, clearsky_model='ineichen', aoi_model='no_loss', spectral_model="no_loss", losses_model='pvwatts')

    return modelChain


def remove_obvious_outliers(measured_series, max_estimated_series):   
    # Remove stong outilers from the measured series, by removing all value that are greater than 2 time the expected value, and less than 0.
    measured_series = measured_series[(measured_series < 3*max_estimated_series.max()) & (measured_series >= 0)]
    return measured_series

def tune_max_production_estimator(measured_series, max_estimated_series, window=7): 
    # Remove the obvious outliers. It's important before calculating the std, which can be strongly impacted by the strong outliers.
    measured_series = remove_obvious_outliers(measured_series, max_estimated_series)

    # Keep only the max measured value
    max_measured_series = pd.Series(index=measured_series.index, dtype=float)
    # Iterate over windows of a given size, and keep only the maximum value in each window
    for i in range(0, len(measured_series), window):
        window_data = measured_series.iloc[i:i+window]
        if not window_data.empty and not window_data.isna().all():
            max_value = window_data.max()
            max_index = window_data.idxmax(skipna=True)
            max_measured_series[max_index] = max_value
    
    # Calculate the relative difference between the maximum measured and maximum estimated value
    realtive_difference = max_measured_series / max_estimated_series

    # Compute statistics
    std = realtive_difference.std()
    mean = realtive_difference.mean()

    # Remove the outilers that have a z-score greater than 1
    z_scores = np.abs(realtive_difference - mean) / std
    realtive_difference = realtive_difference[z_scores < 1]

    # Get the loss that overestimate the estimate maximum daily energy
    loss = 1-realtive_difference.max()
    
    return loss, std






### Create estimator


In [ ]:
cacheFilename_systemsData_EstimatedMaxDailyEnergy = os.path.join(dataCacheDirpath, 'systemsData_EstimatedMaxDailyEnergy.pkl')

if useCached and os.path.exists(cacheFilename_systemsData_EstimatedMaxDailyEnergy):
    # TODO how to deal if the cached data is not up to date and some systems have been added or removed?
    print(f"Loading cached data in {cacheFilename_systemsData_EstimatedMaxDailyEnergy}")
    systemsData_EstimatedMaxDailyEnergy = pd.read_pickle(cacheFilename_systemsData_EstimatedMaxDailyEnergy)
else:
    systemsData_EstimatedMaxDailyEnergy_dic = {}
    for systemName in systemsName_Target:
        tuned  = not tuneMaxProductionEstimators # If we don't want to tune the estimators, we say that the estimator is already tuned
        # reset the loss in the metadata if we want to tune the estimators
        if tuneMaxProductionEstimators:
            systemsMetadata[systemName]['metadata']['loss'] = 0

        while True: # emulate do while loop
            ## ------------------ ##
            ## Create ModelChains ##
            ## ------------------ ##
            estimator = generate_max_production_estimator(systemsMetadata[systemName])

            ## ------------------- ##
            ## Simulate production ##
            ## ------------------- ##
            measured_series = systemsData_MeasuredDailyEnergy[systemName]
            startDate = measured_series[~measured_series.isna()].index.min()
            endDate = measured_series[~measured_series.isna()].index.max()
            estimatedMaxDailyEnergy = generate_max_production_estimate(startDate, endDate, estimator, samplingFreq='1h')

            # add the series to the dictionary
            systemsData_EstimatedMaxDailyEnergy_dic[systemName] = estimatedMaxDailyEnergy

            ## --------------- ##
            ## Tune estimators ##
            ## --------------- ##
            if tuned:
                break

            loss, std = tune_max_production_estimator(measured_series, estimatedMaxDailyEnergy)

            # write the loss in systemsMetadata
            systemsMetadata[systemName]['metadata']['loss'] = loss

            if std > 1:
                systemsName_Valid.remove(systemName)
                systemsName_Target.remove(systemName)
                print(f"System {systemName} : We can't find the model corresponding to the measured data. This system is removed from the list of systems to be processed.")

            tuned = True



    # Concatenate all the columns in the list to create one dataframe
    systemsData_EstimatedMaxDailyEnergy = pd.concat(systemsData_EstimatedMaxDailyEnergy_dic, axis=1)
    systemsData_EstimatedMaxDailyEnergy.index = pd.to_datetime(systemsData_EstimatedMaxDailyEnergy.index)
    systemsData_EstimatedMaxDailyEnergy.sort_index(inplace=True)


    # Save the dataframe to a CSV file
    systemsData_EstimatedMaxDailyEnergy.to_pickle(cacheFilename_systemsData_EstimatedMaxDailyEnergy)

    # Save metadata with tuned parameters
    if tuneMaxProductionEstimators:
        with open(metadataFilepath, 'w') as f:
            json.dump(systemsMetadata, f, indent=4)

# Print the dataframe
systemsData_EstimatedMaxDailyEnergy

### Estimator tuning


In [ ]:
# TODO

### Relative production

True production scaled by the maximum production from the simulator


In [ ]:
# Calculate the relative energy for each system
systemsData_RelativeMeasuredDailyEnergy = systemsData_MeasuredDailyEnergy / systemsData_EstimatedMaxDailyEnergy

### Compare the difference between simulation with hourly and 10min sampling rate


## Half-Sibling Regression


### Functions


In [ ]:

def get_system_data(targetName):
    # Create the feature matrix X and the target vector y
    X = systemsData_RelativeMeasuredDailyEnergy.drop(columns=targetName)
    y = systemsData_RelativeMeasuredDailyEnergy[targetName]
    # remove the observations where their is no target value
    X = X[~y.isna()]
    y = y[~y.isna()]
    return X, y


def mean_absolute_percentage_error_mean_denominator(
    y_true, y_pred, *, sample_weight=None, multioutput="uniform_average"
):
    # Copy of the function mean_absolute_percentage_error from sklearn.metrics._regression, with the denominator of the MAPE changed to the mean of the true values
    import sklearn

    y_type, y_true, y_pred, multioutput = sklearn.metrics._regression._check_reg_targets(
        y_true, y_pred, multioutput
    )
    sklearn.utils.validation.check_consistent_length(y_true, y_pred, sample_weight)
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.mean(np.abs(y_true)), epsilon)
    output_errors = np.average(mape, weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == "raw_values":
            return output_errors
        elif multioutput == "uniform_average":
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)


def mad(arr):
    return abs(arr - arr.median()).median()


def modified_z_score(arr):
    # based on https://www.ibm.com/docs/en/cognos-analytics/11.1.0?topic=terms-modified-z-score
    mad_value = mad(arr)
    if mad_value == 0:
        MeanAD = np.mean(np.abs(arr - np.mean(arr)))
        denominator = 1.253314 * MeanAD
    else:
        denominator = 1.486 * mad_value

    return (arr - np.median(arr)) / denominator


def metrics(y_true, y_pred):
    return {
        'MAPE': mean_absolute_percentage_error(y_true, y_pred),
        'MAPE-MD': mean_absolute_percentage_error_mean_denominator(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': root_mean_squared_error(y_true, y_pred),
        'R2': r2_score(y_true, y_pred)
    }


mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Return the metrics for a RFR model trained on the given data.
# The entire dataset is used for training, and the OOB prediction is used to compute the metrics.


def obb_metrics(X, y, metricFct, rf_parames={}):
    model = RandomForestRegressor(oob_score=True, **rf_parames)
    y_pred = model.fit(X, y).oob_prediction_
    return metricFct(y, y_pred)

# Return the metrics for a RFR model trained on the given data.
# KFold cross-validation is used train the model and to compute the metrics.


def kfold_metrics(X, y, metricFct, rf_parames={}, n_folds=5):
    model = RandomForestRegressor(**rf_parames)
    metrics_list = []

    for train_index, test_index in KFold(n_splits=n_folds).split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        y_pred = model.fit(X_train, y_train).predict(X_test)
        metrics_list.append(metricFct(y_test, y_pred))

    if isinstance(metrics_list[0], dict):
        # Convert list of dictionaries to a DataFrame
        metrics_df = pd.DataFrame(metrics_list)
        # Compute mean for each column
        aggregated_metrics = metrics_df.mean().to_dict()
        return aggregated_metrics
    else:
        # Compute mean of the list for numerical metrics
        return np.mean(metrics_list)


def _accumulate_prediction(predict, X, out, lock):
    """
    This is a utility function for joblib's Parallel.

    It can't go locally in ForestClassifier or ForestRegressor, because joblib
    complains that it cannot pickle it when placed there.
    """
    prediction = predict(X, check_input=False)
    with lock:
        out.append(prediction)


def predict_w_std(self, X):
    """
    Predict regression target and standard deviation for X.

    The predicted regression target of an input sample is computed as the
    mean predicted regression targets of the trees in the forest. The standard
    deviation of the predicted regression targets of the trees in the forest
    is also computed to provide an estimate of the prediction uncertainty.

    Parameters
    ----------
    X : {array-like, sparse matrix} of shape (n_samples, n_features)
        The input samples. Internally, its dtype will be converted to
        ``dtype=np.float32``. If a sparse matrix is provided, it will be
        converted into a sparse ``csr_matrix``.

    Returns
    -------
    mean_predictions : ndarray of shape (n_samples,)
        The predicted values (mean of the predictions from all estimators).
    std_predictions : ndarray of shape (n_samples,)
        The standard deviation of the predicted values (standard deviation of the
        predictions from all estimators).

    Raises
    ------
    NotImplementedError
        If the model was trained for multi-output regression.

    Notes
    -----
    This function does not support multi-output regression. If the model was
    trained for multi-output regression, an exception will be raised.
    """

    if self.n_outputs_ > 1:
        raise NotImplementedError("Variance for multi-output regression is not supported now")

    check_is_fitted(self)
    # Check data
    X = self._validate_X_predict(X)

    # Assign chunk of trees to jobs
    n_jobs, _, _ = _partition_estimators(self.n_estimators, self.n_jobs)

    # avoid storing the output of every estimator by summing them here

    # Initialize a list to collect predictions from each estimator
    all_predictions = []

    # Parallel loop
    lock = threading.Lock()
    Parallel(n_jobs=n_jobs, verbose=self.verbose, require="sharedmem")(
        delayed(_accumulate_prediction)(e.predict, X, all_predictions, lock)
        for e in self.estimators_
    )

    # Convert list to numpy array for easier manipulation
    all_predictions = np.array(all_predictions)

    # Compute mean and variance across predictions from all estimators
    mean_predictions = np.mean(all_predictions, axis=0)
    std_predictions = np.std(all_predictions, axis=0)

    return mean_predictions, std_predictions


RandomForestRegressor.predict_w_std = predict_w_std

### Hyperparameters tuning


#### Hyperparameters tuning


### Train regressors


In [ ]:
import time
serializer = PickleSerializer()

rf_regressors = {}

# Random Forest Regressor hyperparameters
n_estimators = 100  # Number of trees in random forest
max_features = 'log2'  # Number of features to consider at every split
max_depth = None  # Maximum number of levels in tree
min_samples_split = 2  # Minimum number of samples required to split a node
min_samples_leaf = 1  # Minimum number of samples required at each leaf node


if useCached and not forceTrain:
    # load the models in dataCacheDirpath/rf_regressors. The file name is the system name.
    for systemName in systemsName_Target:
        try:
            rf_regressors[systemName] = serializer.retrieve_model(os.path.join(dataCacheDirpath, 'rf_regressors'), systemName)
        except FileNotFoundError:
            continue
    print(f"Loaded {len(rf_regressors)}/{len(systemsName_Target)} models. {len(systemsName_Target) - len(rf_regressors)} models to train.")


for targetName in tqdm(set(systemsName_Target) - set(rf_regressors), desc='Training regressors'):
    rf_regressor = RandomForestRegressor(oob_score=True, random_state=random_state, n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
    X, y = get_system_data(targetName)
    # split the data into training and testing sets
    rf_regressor.fit(X, y)
    rf_regressors[targetName] = rf_regressor
    # save the model in dataCacheDirpath/rf_regressors. The file name is the system name.
    serializer.save_model(rf_regressor, os.path.join(dataCacheDirpath, 'rf_regressors'), targetName)

In [ ]:
targetName = 'a001286'
# Create the feature matrix X and the target vector y
X = systemsData_RelativeMeasuredDailyEnergy.drop(columns=targetName)
y = systemsData_RelativeMeasuredDailyEnergy[targetName]
# remove the observations where their is no target value
X = X[~y.isna()]
y = y[~y.isna()]

# split the data into training and testing sets. data after the 1st June 2023 are used for training
X_test, X_train  = X.loc[X.index < pd.Timestamp('2023-06-01')], X.loc[X.index >= pd.Timestamp('2023-06-01')]
y_test, y_train = y.loc[y.index < pd.Timestamp('2023-06-01')], y.loc[y.index >= pd.Timestamp('2023-06-01')]




rf_regressor = RandomForestRegressor(oob_score=True, random_state=random_state, n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
rf_regressor.fit(X_train, y_train)
y_pred, y_std = rf_regressor.predict_w_std(X_test)

# plot y_test, y_train, and y_pred with error bar y_std with two colors
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test.index, y=y_test, mode='markers', name='y_test'))
fig.add_trace(go.Scatter(x=y_train.index, y=y_train, mode='markers', name='y_train'))
fig.add_trace(go.Scatter(x=y_test.index, y=y_pred, mode='markers', name='y_pred'))
# fig.add_trace(go.Scatter(x=y_test.index, y=y_pred + y_std, mode='lines', line=dict(width=0), showlegend=False))
# fig.add_trace(go.Scatter(x=y_test.index, y=y_pred - y_std, mode='lines', fill='tonexty', fillcolor='rgba(0,100,80,0.2)', line=dict(width=0), showlegend=False))
fig.update_layout(title=f'{targetName} prediction with error bars', xaxis_title='Date', yaxis_title='Relative energy')
fig.show()


In [ ]:
modelChains[targetName]

### Compupte metrics of the model


In [ ]:
cacheFilename_regressorsMetrics = os.path.join(dataCacheDirpath, 'metrics.csv')

if useCached and os.path.exists(cacheFilename_regressorsMetrics):
    # TODO how to deal if the cached data is not up to date and some systems have been added or removed?
    print(f"Loading cached data in {cacheFilename_regressorsMetrics}")
    regressorsMetrics = pd.read_csv(cacheFilename_regressorsMetrics, index_col=0).squeeze()
else:

    regressorsMetrics = pd.Series(index=systemsName_Target, name='MAE')

    for targetName in systemsName_Target:

        X, y = get_system_data(targetName)

        rf_regressor = rf_regressors[targetName]

        regressorsMetrics.loc[targetName] = mean_absolute_error(y, rf_regressor.oob_prediction_)

    # save the metrics

    regressorsMetrics.to_csv(cacheFilename_regressorsMetrics)

### Compute feature importance


In [ ]:
compute_permutation_importance = False

cacheFilename_features_importance = os.path.join(dataCacheDirpath, 'features_importance.csv')
cacheFilename_permutation_importance_mean = os.path.join(dataCacheDirpath, 'permutation_importance_mean.csv')
cacheFilename_permutation_importance_std = os.path.join(dataCacheDirpath, 'permutation_importance_std.csv')

# start = time.time()

if useCached and os.path.exists(cacheFilename_features_importance):
    print(f"Loading cached data in {cacheFilename_features_importance}")
    features_importance_df = pd.read_csv(cacheFilename_features_importance, index_col=0)
else:
    features_importance_df = pd.DataFrame(index=systemsName_Target, columns=systemsName_Valid)
    for targetName in systemsName_Target:
        X, y = get_system_data(targetName)
        rf_regressor = rf_regressors[targetName]
        features_importance_df.loc[targetName, X.columns] = rf_regressor.feature_importances_
    # save the feature importances
    features_importance_df.to_csv(cacheFilename_features_importance)


if compute_permutation_importance:
    if useCached and os.path.exists(cacheFilename_permutation_importance_mean) and os.path.exists(cacheFilename_permutation_importance_std):
        print(f"Loading cached data in {cacheFilename_permutation_importance_mean}")
        permutation_importance_mean_df = pd.read_csv(cacheFilename_permutation_importance_mean, index_col=0)
        print(f"Loading cached data in {cacheFilename_permutation_importance_std}")
        permutation_importance_std_df = pd.read_csv(cacheFilename_permutation_importance_std, index_col=0)
    else:
        permutation_importance_mean_df = pd.DataFrame(index=systemsName_Target, columns=systemsName_Valid)
        permutation_importance_std_df = pd.DataFrame(index=systemsName_Target, columns=systemsName_Valid)
        for targetName in tqdm(systemsName_Target):
            X, y = get_system_data(targetName)
            rf_regressor = rf_regressors[targetName]
            permutation_importance_results = permutation_importance(rf_regressor, X, y, n_repeats=5, random_state=random_state, n_jobs=-1, scoring=mae_scorer)
            permutation_importance_mean_df.loc[targetName, X.columns] = permutation_importance_results.importances_mean
            permutation_importance_std_df.loc[targetName, X.columns] = permutation_importance_results.importances_std
        # save the permutation importances
        permutation_importance_mean_df.to_csv(cacheFilename_permutation_importance_mean)
        permutation_importance_std_df.to_csv(cacheFilename_permutation_importance_std)


# print(f"Time elapsed: {time.time() - start} - Time per system: {(time.time() - start) / len(systemsName_Target)}")

### Generate expected value for each systems


In [ ]:
cacheFilename_systemsData_RelativeExpectedDailyEnergy_mean = os.path.join(dataCacheDirpath, 'systemsData_RelativeExpectedDailyEnergy_mean.pkl')
cacheFilename_systemsData_RelativeExpectedDailyEnergy_std = os.path.join(dataCacheDirpath, 'systemsData_RelativeExpectedDailyEnergy_std.pkl')

if useCached and os.path.exists(cacheFilename_systemsData_RelativeExpectedDailyEnergy_mean) and os.path.exists(cacheFilename_systemsData_RelativeExpectedDailyEnergy_std):

    print(f"Loading cached data in {cacheFilename_systemsData_RelativeExpectedDailyEnergy_mean}")
    systemsData_RelativeExpectedDailyEnergy_mean = pd.read_pickle(cacheFilename_systemsData_RelativeExpectedDailyEnergy_mean)
    print(f"Loading cached data in {cacheFilename_systemsData_RelativeExpectedDailyEnergy_std}")
    systemsData_RelativeExpectedDailyEnergy_std = pd.read_pickle(cacheFilename_systemsData_RelativeExpectedDailyEnergy_std)

else:

    # Create an empty list to store all expected data for each systems

    systemsData_RelativeExpectedDailyEnergy_mean_List = []
    systemsData_RelativeExpectedDailyEnergy_std_List = []

    for systemName in tqdm(systemsName_Target, desc='Generating expected daily energy'):
        # Comute the expected daily energy for the target system for all the dates
        X, _ = get_system_data(systemName)
        y_mean, y_std = rf_regressors[systemName].predict_w_std(X)
        y_mean = pd.Series(y_mean, index=X.index, name=systemName)
        y_std = pd.Series(y_std, index=X.index, name=systemName)
        systemsData_RelativeExpectedDailyEnergy_mean_List.append(y_mean)
        systemsData_RelativeExpectedDailyEnergy_std_List.append(y_std)

    # Concatenate all the columns in the list to create one dataframe
    systemsData_RelativeExpectedDailyEnergy_mean = pd.concat(systemsData_RelativeExpectedDailyEnergy_mean_List, axis=1).sort_index()
    systemsData_RelativeExpectedDailyEnergy_std = pd.concat(systemsData_RelativeExpectedDailyEnergy_std_List, axis=1).sort_index()

    # Save the dataframe to a CSV file
    systemsData_RelativeExpectedDailyEnergy_mean.to_pickle(cacheFilename_systemsData_RelativeExpectedDailyEnergy_mean)
    systemsData_RelativeExpectedDailyEnergy_std.to_pickle(cacheFilename_systemsData_RelativeExpectedDailyEnergy_std)

In [ ]:
# Compute absolute expected daily energy
systemsData_ExpectedDailyEnergy_mean = systemsData_RelativeExpectedDailyEnergy_mean * systemsData_EstimatedMaxDailyEnergy
systemsData_ExpectedDailyEnergy_std = systemsData_RelativeExpectedDailyEnergy_std * systemsData_EstimatedMaxDailyEnergy

# Scaling technics & Outliers removal

Compute the:

- Global mean
- Global median
- Global standard deviation
- Rolling mean
- Rolling median
- Rolling standard deviation
- Simulate max production without info
- SImulated max production with info


# Plot results


In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__)

tab_height = '2em'
app.layout = html.Div([
    html.Div([
        dcc.Dropdown(
            id='system-dropdown',
            options=[{'label': name, 'value': name} for name in systemsName_Target],
            value=systemsName_Target[0],
            style={'width': '50%'}  # Adjust width and font size
        ),
        html.Div(id='metric-text', style={'display': 'inline-block', 'margin-left': '20px', 'fontSize': 16})  # Container for the metric text
    ], style={'display': 'flex', 'align-items': 'center'}),  # Align items horizontally
    dcc.Tabs(id='plot-tabs', value='tab-energy', children=[
        dcc.Tab(label='Energy', value='tab-energy', style={'padding': '0px', 'lineHeight': tab_height}, selected_style={'padding': '0px', 'lineHeight': tab_height, 'fontWeight': 'bold'}),  # Adjust height and line height
        dcc.Tab(label='Relative Energy', value='tab-rel-energy', style={'padding': '0px', 'lineHeight': tab_height}, selected_style={'padding': '0px', 'lineHeight': tab_height, 'fontWeight': 'bold'}),  # Adjust height and line height
        dcc.Tab(label='Similar neighboring systems', value='tab-neighbors', style={'padding': '0px', 'lineHeight': tab_height}, selected_style={'padding': '0px', 'lineHeight': tab_height, 'fontWeight': 'bold'}),  # Adjust height and line height
    ]),  # Adjust height for tabs
    html.Div(id='tabs-content', style={'flex': '1 1 auto'})  # Allow the tabs-content div to grow
], style={'display': 'flex', 'flexDirection': 'column', 'height': '100vh'})  # Make the outer container fill the screen height


@app.callback(
    [Output('tabs-content', 'children'),
     Output('metric-text', 'children')],
    [Input('plot-tabs', 'value'),
     Input('system-dropdown', 'value')]
)
def render_content(tab, selected_system):
    #test if the regressorsMetrics variable exist
    try:
        # Mean Absolute Error: {regressorsMetrics.loc[selected_system] * 100:.2f}\n
        metric_text = f"Loss: {systemsMetadata[selected_system]['metadata']['loss']*100:.2f}%"
    except:
        metric_text = "No metric available for this system"

    if tab == 'tab-energy':
        fig1 = go.Figure(layout_yaxis_title="Daily Energy (kWh)")

        try:
            fig1.add_trace(go.Scatter(
                x=systemsData_EstimatedMaxDailyEnergy[selected_system].index,
                y=systemsData_EstimatedMaxDailyEnergy[selected_system],
                mode='markers',
                name='Simulated Max Daily Energy'
            ))
        except: pass
        
        # try:
        #     fig1.add_trace(go.Scatter(
        #         x=systemsData_EstimatedMaxDailyEnergy2[selected_system].index,
        #         y=systemsData_EstimatedMaxDailyEnergy2[selected_system],
        #         mode='markers',
        #         name='Simulated Max Daily Energy TUNED'
        #     ))
        # except: pass

        try:
            fig1.add_trace(go.Scatter(
                x=systemsData_MeasuredDailyEnergy[selected_system].index,
                y=systemsData_MeasuredDailyEnergy[selected_system],
                mode='markers',
                name='Measured Daily Energy'
            ))
        except: pass

        try:
            fig1.add_trace(go.Scatter(
                x=systemsData_ExpectedDailyEnergy_mean[selected_system].index,
                y=systemsData_ExpectedDailyEnergy_mean[selected_system],
                mode='markers',
                name='Expected Daily Energy'
            ))
        except: pass

        # Update layout for legend position
        fig1.update_layout(
            legend=dict(
                x=0.99,
                y=0.99,
                xanchor='right',
                yanchor='top',
                orientation='h'
            )
        )

        return dcc.Graph(figure=fig1, style={'height': '100%', 'width': '100%'}), metric_text  # Adjust height and width of the figure

    elif tab == 'tab-rel-energy':
        fig1 = go.Figure(layout_yaxis_title="Proportional Daily Energy (%)")
        try:
            fig1.add_trace(go.Scatter(
                x=systemsData_RelativeMeasuredDailyEnergy[selected_system].index,
                y=systemsData_RelativeMeasuredDailyEnergy[selected_system],
                mode='markers',
                name='Measured Daily Energy'
            ))
        except: pass
        try:
            fig1.add_trace(go.Scatter(
                x=systemsData_RelativeExpectedDailyEnergy_mean[selected_system].index,
                y=systemsData_RelativeExpectedDailyEnergy_mean[selected_system],
                mode='markers',
                name='Expected Daily Energy'
            ))
        except: pass

        return dcc.Graph(figure=fig1, style={'height': '100%', 'width': '100%'}), metric_text  # Adjust height and width of the figure

    elif tab == 'tab-neighbors':
        fig2 = go.Figure()

        # Add initial traces with secondary y-axis
        try:
            fig2.add_trace(go.Bar(
                x=features_importance_df.columns,
                y=features_importance_df.loc[selected_system],
                name='Impurity-based Importance',
                yaxis='y1',
                offsetgroup=1
            ))
        except: pass
        try:
            fig2.add_trace(go.Bar(
                x=permutation_importance_mean_df.columns,
                y=permutation_importance_mean_df.loc[selected_system],
                name='Permutation Importance',
                yaxis='y2',
                offsetgroup=2
            ))
        except: pass
        try:
            fig2.update_layout(
                yaxis1=dict(
                    title='Impurity-based Importance',
                    range=[0, features_importance_df.loc[selected_system].max()],
                )
            )
        except: pass
        try:
            fig2.update_layout(
                yaxis2=dict(
                    title='Permutation Importance',
                    overlaying='y',
                    side='right',
                    range=[0, permutation_importance_mean_df.loc[selected_system].max()],
                )
            )
        except: pass

        return dcc.Graph(figure=fig2, style={'height': '100%', 'width': '100%'}), metric_text  # Adjust height and width of the figure


def open_browser():
    webbrowser.open("http://127.0.0.1:8050/")


if __name__ == '__main__':
    # Open the Dash app in a new browser window
    Timer(1, open_browser).start()
    app.run_server(debug=True, use_reloader=False)

## Interpreting result from a prediction

https://towardsdatascience.com/interpreting-random-forests-638bca8b49ea


In [ ]:
systemsMetadata['a001464']['arrays']